# 予備実験「主張を述べる文章と根拠を述べる文章のTransformer分類器」
- [参考：hands-on-ml 16章 Sentiment Analysys](https://github.com/ageron/handson-ml2/blob/master/16_nlp_with_rnns_and_attention.ipynb)

## セットアップ

### Googleドライブのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### hands-on-ml 16章

In [2]:
# # Python ≥3.5 is required
# import sys
# assert sys.version_info >= (3, 5)

# # Is this notebook running on Colab or Kaggle?
# IS_COLAB = "google.colab" in sys.modules
# IS_KAGGLE = "kaggle_secrets" in sys.modules

# if IS_COLAB:
#     !pip install -q -U tensorflow-addons
#     !pip install -q -U transformers

# # Scikit-Learn ≥0.20 is required
# import sklearn
# assert sklearn.__version__ >= "0.20"

# # TensorFlow ≥2.0 is required
# import tensorflow as tf
# from tensorflow import keras
# assert tf.__version__ >= "2.0"

# if not tf.config.list_physical_devices('GPU'):
#     print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
#     if IS_COLAB:
#         print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
#     if IS_KAGGLE:
#         print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# # to make this notebook's output stable across runs
# np.random.seed(42)
# tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## データのインポート・前処理

### データセット概要
- [IBM Debater® - Claims and Evidence (EMNLP 2015)](https://www.research.ibm.com/haifa/dept/vst/debating_data.shtml#Download)
    - 作品がCC-BY-SAでリリースされていることを示すライセンス表示
        - 権利者の名前を入れる
        - 加工しても同じライセンスで他人にもシェア
        - 商用利用OK
    - a)ライセンスのテキストへのハイパーリンクまたはURL
    - またはb)ライセンスのコピーのいずれかを含める
    - 文末のカンマやクエスチョンマークは除去済み

### インポートとテキスト処理
- 変数
    - claims_data
        - claims.txtの分割した行のリスト
    - evidence_data
        - evidence.txtの分割した行のリスト
    - claims
        - 主張の文のリスト
    - evidence
        - 根拠の文のリスト
- 前処理
    - 改行文字の除去
    - タブ文字で分割
    - 大文字を小文字に変換

In [3]:
CLAIMS_DIR = "/content/drive/MyDrive/lab/experiment/transformer-classifier-as-claim-or-evidence/IBM_Debater_(R)_CE-EMNLP-2015.v3/claims.txt"
EVIDENCE_DIR = "/content/drive/MyDrive/lab/experiment/transformer-classifier-as-claim-or-evidence/IBM_Debater_(R)_CE-EMNLP-2015.v3/evidence.txt"
CLAIMS_IDX = 2
EVIDENCE_IDX = 2

claims_data = []
evidence_data = []
claims = []
evidence = []

# read
with open(CLAIMS_DIR, "r", encoding="utf_8") as f:
    lines = f.readlines()
    print("claims.txt lines:")
    print(lines[0])
    print(lines[1])
    print(lines[2])
    print()
    for line in lines[1:]:
        fields = line.rstrip("\n").split("\t")
        claims_data.append(fields)
        claims.append(fields[CLAIMS_IDX])

with open(EVIDENCE_DIR, "r", encoding="utf_8") as f:
    lines = f.readlines()
    print("evidence.txt lines:")
    print(lines[0])
    print(lines[1])
    print(lines[2])
    print()
    for line in lines:
        fields = line.rstrip("\n").split("\t")
        evidence_data.append(fields)
        evidence.append(fields[EVIDENCE_IDX].lower())

# print debug
print("claims_data:")
print(claims_data[0])
print(claims_data[1])
print()

print("evidence_data:")
print(evidence_data[0])
print(evidence_data[1])
print()

print("claims:")
print(claims[0])
print(claims[1])
print()

print("evidence:")
print(evidence[0])
print(evidence[1])

claims.txt lines:
Topic	Claim original text	Claim corrected version

This house believes that the sale of violent video games to minors should be banned	Exposure to violent video games causes at least a temporary increase in aggression and this exposure correlates with aggression in the real world	exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world

This house believes that the sale of violent video games to minors should be banned	video game violence is not related to serious aggressive behavior in real life	video game violence is not related to serious aggressive behavior in real life


evidence.txt lines:
This house believes that the sale of violent video games to minors should be banned	Exposure to violent video games causes at least a temporary increase in aggression and this exposure correlates with aggression in the real world	A 2001 study found that exposure to violent video games

### ラベリング

In [4]:
CLAIMS_LABEL = 1
EVIDENCE_LABEL = 0

if (CLAIMS_LABEL):
    claims_labels = np.ones(len(claims)).tolist()
else:
    claims_labels = np.zeros(len(claims)).tolist()

if (EVIDENCE_LABEL):
    evidence_labels = np.ones(len(evidence)).tolist()
else:
    evidence_labels = np.zeros(len(evidence)).tolist()

print(claims_labels[0])
print(evidence_labels[0])

1.0
0.0


In [5]:
claims_dataset = []
evidence_dataset = []

for idx in range(len(claims)):
    claims_dataset.append([claims[idx], claims_labels[idx]])

for idx in range(len(evidence)):
    evidence_dataset.append([evidence[idx], evidence_labels[idx]])

print(claims_dataset[0])
print(evidence_dataset[0])

['exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world', 1.0]
['a 2001 study found that exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world', 0.0]


### 結合

In [6]:
dataset = claims_dataset + evidence_dataset
print(dataset[0])
print(dataset[-1])

['exposure to violent video games causes at least a temporary increase in aggression and that this exposure correlates with aggression in the real world', 1.0]
['in 2007, professor larry j. sabato revived the debate over term limits by arguing in a more perfect constitution that the success and popularity of term limits at the state level suggests that they should be adopted at the federal level as well', 0.0]


### シャッフル

In [7]:
import random
random.seed(10)

random.shuffle(dataset)
print(dataset[0])
print(dataset[-1])

['a joint research project of the wipo and the united nations university measuring the impact of ip systems on six asian countries found "a positive correlation between the strengthening of the ip system and subsequent economic growth."[ref', 0.0]
['eric posner, international law professor at the university of chicago law school, noting that the raid had "led to wild accusations of illegality", wrote that blockades are lawful during times of armed conflict (such as the coalition blockade of iraq during the first gulf war), and that "war-like conditions certainly exist between israel and hamas" [ref]. he compared israel\'s blockade to the union blockade by the union against the confederacy (a non-state) during the u.s. civil war [ref]. the u.s. supreme court later affirmed the legitimacy of that blockade [ref', 0.0]


### 訓練、評価、テストデータに分割

In [8]:
# VALID_RATE = 0.2
# TEST_RATE = 0.2

# dataset_num = len(dataset)
# valid_num = int(dataset_num * VALID_RATE)
# test_num = int(dataset_num * TEST_RATE)
# train_num = dataset_num - valid_num - test_num

# valid_dataset = dataset[:valid_num]
# test_dataset = dataset[valid_num:valid_num + test_num]
# train_dataset = dataset[valid_num + test_num:]

# print("dataset_num: ", dataset_num)
# print("valid_num: ", valid_num)
# print("test_num: ", test_num)
# print("train_num: ", train_num)

In [9]:
EVAL_RATE = 0.2

dataset_num = len(dataset)
eval_num = int(dataset_num * EVAL_RATE)
train_num = dataset_num - eval_num

eval_dataset = dataset[:eval_num]
train_dataset = dataset[eval_num:]

print("dataset_num: ", dataset_num)
print("eval_num: ", eval_num)
print("train_num: ", train_num)

dataset_num:  6986
eval_num:  1397
train_num:  5589


## 分類器の作成
- [Simple Transformers 入門 (1) - テキスト分類 ](https://note.com/npaka/n/nfe2436ea5301#9VV1a)
- [Simple Transformers 入門 (10) - ハイパーパラメータの最適化 ](https://note.com/npaka/n/n298f269c2275)

### Simple Transformerのインストール

In [10]:
!pip install transformers
!pip install simpletransformers

  Using cached https://files.pythonhosted.org/packages/56/aa/4ef5aa67a9a62505db124a5cb5262332d1d4153462eb8fd89c9fa41e5d92/urllib3-1.25.11-py2.py3-none-any.whl
ERROR: wandb 0.11.0 has requirement urllib3>=1.26.5, but you'll have urllib3 1.25.11 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 6.0.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.25.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6
  Using cached https://files.pythonhosted.org/packages/5f/64/43575537846896abac0b15c3e5ac678d787a4021e906703f1766bfb8ea11/urllib3-1.26.6-py2.py3-none-any.whl
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.6 whic

### モデルの評価まで

In [11]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2021-07-18 21:10:24.051753: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [12]:
# ログの設定
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [13]:
# 学習データの作成
train_df = pd.DataFrame(train_dataset)

# 評価データの作成
eval_df = pd.DataFrame(eval_dataset)

In [14]:
# モデルの作成
model = ClassificationModel('roberta', 'roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out

In [16]:
!rm -rf outputs/

In [17]:
# 学習
model.train_model(train_df, overwrite_output_dir=True)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:616: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5589 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/699 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(699, 0.09424080054817964)

In [18]:
# 評価
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1397 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/175 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.985498034416286, 'tp': 460, 'tn': 928, 'fp': 3, 'fn': 6, 'auroc': 0.9997522162241901, 'auprc': 0.9995180151950078, 'eval_loss': 0.038037594607864905}


In [20]:
# スコア計算
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1397 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/175 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.985498034416286, 'tp': 460, 'tn': 928, 'fp': 3, 'fn': 6, 'auroc': 0.9997522162241901, 'auprc': 0.9995180151950078, 'eval_loss': 0.038037594607864905, 'acc': 0.9935576234788833}


## 英訳した日本記事でのテスト

### データのインポート
- [ColabにKaggleのデータをダウンロードする](https://qiita.com/fastso/items/43e85fd51d6426d14dd7)
- [Japanese FakeNews Dataset]()
    - [オープンデータコモンズパブリックドメイン専用およびライセンス（PDDL）v1.0](http://translate.google.com/translate?hl=ja&sl=auto&tl=ja&u=https%3A%2F%2Fopendatacommons.org%2Flicenses%2Fpddl%2F1-0%2F)
        - 受信者は、本作品を商業的に利用したり、技術的な保護手段を用いたり、本データやデータベースを他のデータベースやデータと組み合わせたり、変更や追加を共有したり、秘密にしたりすることができます。
    - 本物はCC BYのウィキニュース

In [24]:
!pip install kaggle

In [25]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [29]:
!kaggle datasets download -d tanreinama/japanese-fakenews-dataset

  0% 0.00/7.68M [00:00<?, ?B/s]
100% 7.68M/7.68M [00:00<00:00, 71.1MB/s]


In [32]:
!unzip "/content/japanese-fakenews-dataset.zip"

Archive:  /content/japanese-fakenews-dataset.zip
  inflating: fakenews.csv            


### データの前処理

In [39]:
FAKENEWS_DIR = "/content/fakenews.csv"

CONTEXT_IDX = 1
ISFAKE_IDX = 2
NEWSTXT_LABEL = 0

fakenews_data = []
wikinews_txts = []
wikinews_sentences = []

# read
with open(FAKENEWS_DIR, "r", encoding="utf_8") as f:
    lines = f.readlines()
    print("fakenews.csv lines:")
    print(lines[0])
    print(lines[1])
    print(lines[2])
    print()
    for line in lines[1:]:
        fields = line.rstrip("\n").split(",")
        fakenews_data.append(fields)
        if fields[ISFAKE_IDX] == str(NEWSTXT_LABEL): 
            wikinews_txts.append(fields[CONTEXT_IDX])
            wikinews_sentences.append(fields[CONTEXT_IDX].split("。")[:-1])

# print debug
print("fakenews_data:")
print(fakenews_data[0])
print(fakenews_data[1])
print()

print("wikinews_txts:")
print(wikinews_txts[0])
print(wikinews_txts[1])
print()

print("wikinews_sentences:")
print(wikinews_sentences[0])
print(wikinews_sentences[1])
print()

fakenews.csv lines:
id,context,isfake,nchar_real,nchar_fake

000128042337,朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コンビニエンスストア「ローソン」でのサービス提供の差し止めなどを求めていた訴訟で、2006年1月19日、東京地方裁判所でヤマト運輸の請求を棄却する判決が下された。2004年のローソンでの郵便小包サービス「ゆうパック」の受付業務開始に際し、ヤマト運輸は「独占禁止法に違反する不当な廉売」として、日本郵政公社を相手取り、サービス提供の差し止めなどを求めていた。朝日新聞によれば、提訴の内容は、2004年11月のローソンでの「ゆうパック」の受付サービス提供の開始に関連し、租税などの優遇措置を受けている日本郵政公社が、配送料金（運賃）などの有利な取引条件でローソンで「ゆうパック」を開始させたのは、独占禁止法の不当廉売に当たり、ヤマト運輸の利益を侵害されるとして、「ゆうパック」サービス提供の差し止めなどを求めていたもの。朝日新聞によれば、判決内容はヤマト運輸の主張を全面的に否定しており、今後の「ゆうパック」サービスの拡大に弾みがつくものと考えられる。日本郵政公社は、公正妥当な判決とのコメントを出した。一方、ヤマト運輸は、高等裁判所への控訴など、今後の対応については検討するとアナウンスしている。,0,541,0

00012b7a8314,11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置することから、人身売買により、環境問題に加え、環境保護にも関心が向けられた。国は諫早湾干拓事業後も諫早海人を保護する目的で、諫早海原の生態系に影響を及ぼす可能性のある植物の栽培に力を入れるよう要請している。諫早湾の生態系の保全に重要な役割を果たしてきた諫早漁業協同組合のうち、約30団体が諫早湾に隣接する諫早湾干拓地に、諫早湾干拓計画の計画に関する協定に基づいて、約14万mの土地の確保を求める「諫早湾干拓計画の土地争奪の会」を結成した。組合理事長には諫早漁業協同組合長で、諫早干拓地に漁業協定を締結し、2017年(平成29年)2月5日に、干拓地の土地購入を求める請願書を諫早海人の保護に向けて請願書を添えて諫早湾干拓地に対して「諫

### DeepL 翻訳
- [DeepL APIをPythonから利用して日本語の文章を翻訳する](https://deepblue-ts.co.jp/nlp/deepl-api-python/)

In [40]:
!pip install requests

  Using cached https://files.pythonhosted.org/packages/56/aa/4ef5aa67a9a62505db124a5cb5262332d1d4153462eb8fd89c9fa41e5d92/urllib3-1.25.11-py2.py3-none-any.whl
ERROR: wandb 0.11.0 has requirement urllib3>=1.26.5, but you'll have urllib3 1.25.11 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 6.0.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.25.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6


In [47]:
import time
import requests

DEEPL_API_KEY = ''
TRANSLATE_NUM = 10
SLEEP_SEC = 3

wikinews_txts_en = []

for txt_idx in range (TRANSLATE_NUM):
    params = {
            "auth_key": DEEPL_API_KEY,
            "text": wikinews_txts[txt_idx],
            "source_lang": 'JA',
            "target_lang": 'EN'
    }
    request = requests.post("https://api-free.deepl.com/v2/translate", data=params)
    result = request.json()
    # print(result)
    wikinews_txts_en.append(result["translations"][0]["text"])
    time.sleep(SLEEP_SEC)

print(wikinews_txts_en[0])

According to the Asahi Shimbun and other media reports, on January 19, 2006, the Tokyo District Court dismissed a lawsuit filed by Yamato Transport Co. On January 19, 2006, the Tokyo District Court dismissed Yamato Transport's lawsuit, saying that the company had violated the Anti-Monopoly Law by offering "unfairly low prices" when it began accepting "Yu-Pack" postal parcels at Lawson's in 2004. According to the Asahi Shimbun, the lawsuit alleges that, in relation to the launch of the Yu-pack service at Lawson in November 2004, the fact that Japan Post, which receives preferential tax treatment, allowed Lawson to launch the Yu-pack service under favorable trading conditions, including delivery rates, constitutes unfair low sales under the Antimonopoly Law. According to the Asahi Shimbun, the court ruled that Japan Post Co. According to the Asahi Shimbun, the content of the ruling completely denies Yamato Transport's claims, which is expected to give momentum to the expansion of the Yu-

### カンマで分割

In [55]:
wikinews_sentences_en = []

for idx in range(len(wikinews_txts_en)):
    wikinews_sentences_en.append(wikinews_txts_en[idx].split(".")[:-1])

print("wikinews_sentences_en:")
print(wikinews_sentences_en[0])
print(wikinews_sentences_en[1])

wikinews_sentences_en:
['According to the Asahi Shimbun and other media reports, on January 19, 2006, the Tokyo District Court dismissed a lawsuit filed by Yamato Transport Co', ' On January 19, 2006, the Tokyo District Court dismissed Yamato Transport\'s lawsuit, saying that the company had violated the Anti-Monopoly Law by offering "unfairly low prices" when it began accepting "Yu-Pack" postal parcels at Lawson\'s in 2004', ' According to the Asahi Shimbun, the lawsuit alleges that, in relation to the launch of the Yu-pack service at Lawson in November 2004, the fact that Japan Post, which receives preferential tax treatment, allowed Lawson to launch the Yu-pack service under favorable trading conditions, including delivery rates, constitutes unfair low sales under the Antimonopoly Law', ' According to the Asahi Shimbun, the court ruled that Japan Post Co', " According to the Asahi Shimbun, the content of the ruling completely denies Yamato Transport's claims, which is expected to gi

## 分類器へ代入

In [61]:
predicts = []

for article_idx in range(len(wikinews_sentences_en)):
    each_articles_preds = []
    for sentence in wikinews_sentences_en[article_idx]:
        # print(sentence)
        # print(model.predict([sentence]))
        pred = model.predict([sentence])
        each_articles_preds.append(pred)
    predicts.append(each_articles_preds)

print(predicts)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[(array([0]), array([[ 4.76171875, -4.57421875]])), (array([0]), array([[ 4.74609375, -4.5703125 ]])), (array([0]), array([[ 4.80078125, -4.62109375]])), (array([0]), array([[ 4.6875, -4.5625]])), (array([0]), array([[ 4.6015625, -4.515625 ]])), (array([0]), array([[ 4.65625   , -4.50390625]])), (array([1]), array([[-3.67773438,  3.2734375 ]]))], [(array([0]), array([[ 4.734375 , -4.5859375]])), (array([0]), array([[ 4.74609375, -4.60546875]])), (array([0]), array([[ 2.46484375, -2.83203125]])), (array([0]), array([[ 4.69921875, -4.5703125 ]])), (array([0]), array([[ 1.66113281, -1.99023438]])), (array([0]), array([[ 4.55078125, -4.44921875]])), (array([0]), array([[ 4.55859375, -4.46484375]])), (array([1]), array([[-3.62109375,  3.16210938]]))], [(array([0]), array([[ 4.4453125 , -4.34765625]])), (array([1]), array([[-0.28222656,  0.06164551]])), (array([0]), array([[ 4.59375 , -4.546875]])), (array([1]), array([[-1.25976562,  0.81689453]])), (array([0]), array([[ 4.390625, -4.359375

In [69]:
print(predicts[0])
for pred in predicts[0]:
    print(pred)

print(predicts[0][0][0])
print(predicts[0][0][0][0])

[(array([0]), array([[ 4.76171875, -4.57421875]])), (array([0]), array([[ 4.74609375, -4.5703125 ]])), (array([0]), array([[ 4.80078125, -4.62109375]])), (array([0]), array([[ 4.6875, -4.5625]])), (array([0]), array([[ 4.6015625, -4.515625 ]])), (array([0]), array([[ 4.65625   , -4.50390625]])), (array([1]), array([[-3.67773438,  3.2734375 ]]))]
(array([0]), array([[ 4.76171875, -4.57421875]]))
(array([0]), array([[ 4.74609375, -4.5703125 ]]))
(array([0]), array([[ 4.80078125, -4.62109375]]))
(array([0]), array([[ 4.6875, -4.5625]]))
(array([0]), array([[ 4.6015625, -4.515625 ]]))
(array([0]), array([[ 4.65625   , -4.50390625]]))
(array([1]), array([[-3.67773438,  3.2734375 ]]))
[0]
0


### 文章と分類結果の対応

In [71]:
for article_idx in range(len(wikinews_sentences_en)):
    each_articles_preds = []
    print("\n******************************")
    print("article_idx: ", article_idx)
    print("******************************\n")
    for sentence_idx in range(len(wikinews_sentences_en[article_idx])):
        sentence = wikinews_sentences_en[article_idx][sentence_idx]
        if predicts[article_idx][sentence_idx][0][0] == CLAIMS_LABEL:
            print("claim   : ", sentence)
        else:
            print("evidence: ", sentence)


******************************
article_idx:  0
******************************

evidence:  According to the Asahi Shimbun and other media reports, on January 19, 2006, the Tokyo District Court dismissed a lawsuit filed by Yamato Transport Co
evidence:   On January 19, 2006, the Tokyo District Court dismissed Yamato Transport's lawsuit, saying that the company had violated the Anti-Monopoly Law by offering "unfairly low prices" when it began accepting "Yu-Pack" postal parcels at Lawson's in 2004
evidence:   According to the Asahi Shimbun, the lawsuit alleges that, in relation to the launch of the Yu-pack service at Lawson in November 2004, the fact that Japan Post, which receives preferential tax treatment, allowed Lawson to launch the Yu-pack service under favorable trading conditions, including delivery rates, constitutes unfair low sales under the Antimonopoly Law
evidence:   According to the Asahi Shimbun, the court ruled that Japan Post Co
evidence:   According to the Asahi Shimbun

In [77]:
for article_idx in range(len(wikinews_sentences_en)):
    each_articles_preds = []
    print("\n******************************")
    print("article_idx: ", article_idx)
    print("******************************\n")
    for sentence in wikinews_sentences[article_idx]:
        print(sentence)
    print()
    for sentence_idx in range(len(wikinews_sentences_en[article_idx])):
        sentence = wikinews_sentences_en[article_idx][sentence_idx]
        if predicts[article_idx][sentence_idx][0][0] == CLAIMS_LABEL:
            print("claim   : ", sentence)
        else:
            print("evidence: ", sentence)


******************************
article_idx:  0
******************************

朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コンビニエンスストア「ローソン」でのサービス提供の差し止めなどを求めていた訴訟で、2006年1月19日、東京地方裁判所でヤマト運輸の請求を棄却する判決が下された
2004年のローソンでの郵便小包サービス「ゆうパック」の受付業務開始に際し、ヤマト運輸は「独占禁止法に違反する不当な廉売」として、日本郵政公社を相手取り、サービス提供の差し止めなどを求めていた
朝日新聞によれば、提訴の内容は、2004年11月のローソンでの「ゆうパック」の受付サービス提供の開始に関連し、租税などの優遇措置を受けている日本郵政公社が、配送料金（運賃）などの有利な取引条件でローソンで「ゆうパック」を開始させたのは、独占禁止法の不当廉売に当たり、ヤマト運輸の利益を侵害されるとして、「ゆうパック」サービス提供の差し止めなどを求めていたもの
朝日新聞によれば、判決内容はヤマト運輸の主張を全面的に否定しており、今後の「ゆうパック」サービスの拡大に弾みがつくものと考えられる
日本郵政公社は、公正妥当な判決とのコメントを出した
一方、ヤマト運輸は、高等裁判所への控訴など、今後の対応については検討するとアナウンスしている
evidence:  According to the Asahi Shimbun and other media reports, on January 19, 2006, the Tokyo District Court dismissed a lawsuit filed by Yamato Transport Co
evidence:   On January 19, 2006, the Tokyo District Court dismissed Yamato Transport's lawsuit, saying that the company had violated the Anti-Monopoly Law by offering "unfairly low prices" when it began accepting "Yu-

### ラベルの可視化